# 9 - The discrete de Rham sequence

In this tutorial we will learn the application of operators in the deRham diagram (below) to objects of the discrete deRham spaces. The involved data structures have been discussed in [Tutorial 08 - Struphy data structures](https://struphy.pages.mpcdf.de/struphy/tutorials/tutorial_08_data_structures.html?highlight=data%20structures).

The basics of the 3d DeRham diagram are explained in the [struphy documentation](https://struphy.pages.mpcdf.de/struphy/sections/discretization.html#geometric-finite-elements-feec).

![hi](../pics/derham_complex.png)

The discrete complex in the above diagram (lower row) is loaded via the **Derham** class: 

In [ ]:
from mpi4py import MPI
from struphy.psydac_api.psydac_derham import Derham

Nel = [9, 9, 10]  # Number of grid cells
p = [1, 2, 3]  # spline degrees
spl_kind = [False, True, True]   # spline types (clamped vs. periodic)

comm = MPI.COMM_WORLD
derham = Derham(Nel, p, spl_kind, comm=comm)

Let us inspect the properties of `Derham`:

In [ ]:
import numpy as np

for k in dir(derham):
    if k[0] != '_':
        v = getattr(derham, k)
        if isinstance(v, np.ndarray):
            v = f'{type(getattr(derham, k))} of shape {v.shape}'
        print(k.ljust(26), v)

Let us create an element of each discrete space $V_h^n$:

In [ ]:
p0 = derham.create_field('pressure', 'H1')
e1 = derham.create_field('e_field', 'Hcurl')
b2 = derham.create_field('b_field', 'Hdiv')
n3 = derham.create_field('density', 'L2')

Moreover, let us initialize these fields in the following way:

* The `pressure` and the 1st component of `e_field` are sinusoidal functions of mode number 2 and amplitude 0.5 in the third direction
* The 3rd component of `e_field` and the 2nd component of `b_field` are superpositions of two cosines with mode numbers 1 and 2 and amplitudes 0.75 and 0.5, , respectively, in the second direction
* The `density` has noise of amplitude $10^{-3}$ in the third direction
* all other components are zero

In [ ]:
init_params = {'type': ['ModesSin', 'ModesCos', 'noise'],
               'ModesSin': {'coords': 'logical',
                            'comps': {'pressure': [True],
                                      'e_field': [True, False, False]},
                            'ls': [0],
                            'ms': [0],
                            'ns': [2],
                            'amps': [.5]},
               'ModesCos': {'coords': 'logical',
                            'comps': {'e_field': [False, False, True],
                                      'b_field': [False, True, False]},
                            'ls': [0, 0],
                            'ms': [1, 2],
                            'ns': [0, 0],
                            'amps': [.75, .5]},
               'noise': {'comps': {'density': [True]},
                        'variation_in': 'e3',
                        'amp': 0.001,
                        'seed': 3456546}}

p0.initialize_coeffs(init_params)
e1.initialize_coeffs(init_params)
b2.initialize_coeffs(init_params)
n3.initialize_coeffs(init_params)

Let us evaluate these fields, squeeze the output and plot all components for verification:

In [ ]:
from matplotlib import pyplot as plt

# evaluation points
eta1 = 0
eta2 = np.linspace(0., 1., 50)
eta3 = np.linspace(0., 1., 70)

# evaluate 0-form
p0_vals = p0(eta1, eta2, eta3, squeeze_output=True)
print(f'{type(p0_vals) = }, {p0_vals.shape = }')

# evaluate 1-form
e1_vals = e1(eta1, eta2, eta3, squeeze_output=True)
print(f'{type(e1_vals) = }, {type(e1_vals[0]) = }, {e1_vals[0].shape = }')

# evaluate 2-form
b2_vals = b2(eta1, eta2, eta3, squeeze_output=True)
print(f'{type(b2_vals) = }, {type(b2_vals[0]) = }, {b2_vals[0].shape = }')

# evaluate 3-form
n3_vals = n3(eta1, eta2, eta3, squeeze_output=True)
print(f'{type(n3_vals) = }, {n3_vals.shape = }')

# plotting
plt.figure(figsize=(12, 14))
plt.subplot(4, 3, 1)
plt.plot(eta3, p0_vals[0, :], label=p0.name)
plt.xlabel('$\eta_3$')
plt.legend()

plt.subplot(4, 3, 4)
plt.plot(eta3, e1_vals[0][0, :], label=(e1.name + '_1'))
plt.xlabel('$\eta_3$')
plt.legend()
plt.subplot(4, 3, 5)
plt.plot(eta3, e1_vals[1][0, :], label=(e1.name + '_2'))
plt.xlabel('$\eta_3$')
plt.legend()
plt.subplot(4, 3, 6)
plt.plot(eta2, e1_vals[2][:, 0], label=(e1.name + '_3'))
plt.xlabel('$\eta_2$')
plt.legend()

plt.subplot(4, 3, 7)
plt.plot(eta2, b2_vals[0][:, 0], label=(b2.name + '_1'))
plt.xlabel('$\eta_2$')
plt.legend()
plt.subplot(4, 3, 8)
plt.plot(eta2, b2_vals[1][:, 0], label=(b2.name + '_2'))
plt.xlabel('$\eta_2$')
plt.legend()
plt.subplot(4, 3, 9)
plt.plot(eta2, b2_vals[2][:, 0], label=(b2.name + '_3'))
plt.xlabel('$\eta_2$')
plt.legend()

plt.subplot(4, 3, 10)
plt.plot(eta3, n3_vals[0, :], label=n3.name)
plt.xlabel('$\eta_3$')
plt.legend()

Nex, we shall project a sinusoidal function into $V_h^0$ and, moreover, project its gradient into $V_h^1$:

In [ ]:
def fun(x, y, z): return .5*np.sin(2*2*np.pi*z)

fun_h = derham.P['0'](fun)
print(f'{type(fun_h) = }')

def dx_fun(x, y, z): return 0*z
def dy_fun(x, y, z): return 0*z
def dz_fun(x, y, z): return 2*2*np.pi*.5*np.cos(2*2*np.pi*z)

dfun_h = derham.P['1']((dx_fun, dy_fun, dz_fun))
print(f'{type(dfun_h) = }')

We can check the commuting property by applying the discrete gradient operator.

In [ ]:
print(f'{type(derham.grad) = }')
gradfun_h = derham.grad.dot(fun_h)
print(f'{type(gradfun_h) = }')

assert np.allclose(dfun_h[0].toarray(), gradfun_h[0].toarray())
assert np.allclose(dfun_h[1].toarray(), gradfun_h[1].toarray())
assert np.allclose(dfun_h[2].toarray(), gradfun_h[2].toarray())

**All these operations also work in parallel!**